In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key= os.getenv('GOOGLE_API_KEY')
if not api_key :
    raise ValueError('GOOGLE_API_KEY not found!')

In [2]:
import os
import glob
from langchain_core.documents import Document

# Đường dẫn thư mục Dataset
DATASET_DIR = './Dataset_economy'

# Đọc tất cả file .txt từ Dataset_economy (bao gồm các thư mục con)
documents = []
for filepath in glob.glob(os.path.join(DATASET_DIR, '**', '*.txt'), recursive=True):
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            content = f.read().strip()
        if content:  # Bỏ qua file rỗng
            # Lấy tên danh mục từ đường dẫn thư mục
            parts = filepath.replace(DATASET_DIR, '').strip('/').split('/')
            category = parts[0] if len(parts) > 1 else 'unknown'
            
            documents.append(Document(
                page_content=content,
                metadata={
                    "source": filepath,
                    "category": category,
                    "filename": os.path.basename(filepath)
                }
            ))
    except Exception as e:
        print(f"Lỗi đọc file {filepath}: {e}")

print(f"Đã load {len(documents)} văn bản từ {DATASET_DIR}")

Đã load 27682 văn bản từ ./Dataset_economy


In [3]:
from typing import List
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.embeddings import Embeddings


class E5EmbeddingsWrapper(Embeddings):
    def __init__(self, base_embeddings):
        self._base = base_embeddings

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        texts = ["passage: " + t for t in texts]
        return self._base.embed_documents(texts)

    def embed_query(self, text: str) -> List[float]:
        return self._base.embed_query("query: " + text)


# ====== Khởi tạo model ======
model = "intfloat/multilingual-e5-base"

base_embeddings = HuggingFaceEmbeddings(
    model_name=model,
    model_kwargs={"device": "cuda"},  # đổi thành "cpu" nếu cần
    encode_kwargs={
        "normalize_embeddings": True
    }
)

embeddings = E5EmbeddingsWrapper(base_embeddings)

2026-02-27 16:34:58.427849: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-02-27 16:34:58.464027: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-02-27 16:34:59.223590: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [4]:
import os
import shutil
import hashlib
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma.vectorstores import Chroma


PERSIST_DIR = "./chroma_economy_db"
FORCE_REBUILD = True   # Đổi thành False nếu không muốn rebuild
K_RETRIEVE = 40

splitter = RecursiveCharacterTextSplitter(
    chunk_size=600,
    chunk_overlap=80,
    separators=['\n\n', '\n']
)

doc_splits = splitter.split_documents(documents)
print(f"✂️ Tổng số chunks trước dedup: {len(doc_splits)}")

seen_hashes = set()
unique_splits = []

for d in doc_splits:
    content_hash = hashlib.md5(
        d.page_content.strip().encode("utf-8")
    ).hexdigest()

    if content_hash not in seen_hashes:
        seen_hashes.add(content_hash)
        unique_splits.append(d)

removed = len(doc_splits) - len(unique_splits)
doc_splits = unique_splits

print(f"🔄 Dedup: {len(doc_splits) + removed} → {len(doc_splits)} (loại {removed})")


if FORCE_REBUILD and os.path.exists(PERSIST_DIR):
    shutil.rmtree(PERSIST_DIR)
    print(f"🗑️ Đã xóa DB cũ tại {PERSIST_DIR}")

if os.path.exists(PERSIST_DIR):
    print("📂 Đang load DB cũ...")
    chroma = Chroma(
        persist_directory=PERSIST_DIR,
        embedding_function=embeddings
    )
else:
    print("🆕 Đang tạo DB mới...")
    chroma = Chroma.from_documents(
        documents=doc_splits,
        embedding=embeddings,
        persist_directory=PERSIST_DIR
    )

print(f"✅ DB hiện có {chroma._collection.count()} vectors")


retriever_cosine = chroma.as_retriever(
    search_kwargs={'k': K_RETRIEVE}
)

print(f"🔎 Retriever sẵn sàng với k={K_RETRIEVE}")

✂️ Tổng số chunks trước dedup: 306296
🔄 Dedup: 306296 → 120165 (loại 186131)
🆕 Đang tạo DB mới...
✅ DB hiện có 120165 vectors
🔎 Retriever sẵn sàng với k=40


In [5]:
import numpy as np
from scipy.spatial.distance import cdist

def energy_base_distance(X,Y):
    
    X=np.asarray(X)
    Y=np.asarray(Y)
#Tuong tac cheo
    d_xy=cdist(X,Y,metric='euclidean')
    E_xy=np.mean(d_xy)
#Tu nang x
    d_xx=cdist(X,X,metric='euclidean')
    E_xx=np.mean(d_xx)
#Tu nang y
    d_yy=cdist(Y,Y,metric='euclidean')
    E_yy=np.mean(d_yy)
#Energy_distance
    ED=2*E_xy - E_xx - E_yy

    return max(0,ED)

In [6]:
X = [[1,1],[2,3]]
Y = [[1,1],[2,3]]

ed=energy_base_distance(X,Y)
print(f'{ed}')

0


In [7]:
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity

def retrieve_with_energy_clustering(query, retriever, embeddings, k_clusters=7):

    docs=retriever.invoke(query)

    context=[doc.page_content for doc in docs]

    doc_vector= np.array(embeddings.embed_documents(context))
    query_vector=np.array(embeddings.embed_query(query)).reshape(1,-1)
#kiem loi
    sims = cosine_similarity(query_vector, doc_vector)[0]
    print("Max cosine:", np.max(sims))
    if np.max(sims) < 0.40:
        return [] 
#tao cum
    actual_k=min(k_clusters,len(doc_vector))
    kmeans=KMeans(n_clusters= actual_k, random_state=42, n_init='auto')
    labels=kmeans.fit_predict(doc_vector)
#kiem tra e
    best_energy=float('inf')
    best_cluster_idx=-1
    for i in range(actual_k):
        indices=np.where(labels==i)[0]
        cluster_vector=doc_vector[indices]

        energy=energy_base_distance(query_vector,cluster_vector)

        if energy < best_energy:
            best_energy=energy
            best_cluster_idx=i
#tim idx e thap
    win_index=np.where(labels==best_cluster_idx)[0]
    print(f'{best_energy}')
#doan context e thap
    final_docs=[context[i] for i in win_index]

    return final_docs

In [8]:
query='Mục tiêu doanh thu và lợi nhuận năm 2024 của Phục Hưng Holdings (PHC) được đặt ra là bao nhiêu?'
context_doc=retrieve_with_energy_clustering(query,retriever_cosine,embeddings)
print(f'{context_doc}')

Max cosine: 0.8926264617966007
0.6915864902417015
['Nhờ thực hiện tốt công tác quản trị biến động nên mặc dù giá các sản phẩm chủ lực của Tập đoàn 2 tháng đầu năm 2024 giảm từ 3 - 15% so với cùng kỳ năm 2023 nhưng tất cả các chỉ tiêu tài chính của Tập đoàn đều hoàn thành vượt mức từ 18 - 48% kế hoạch, tăng trưởng cao so với cùng kỳ. Tổng doanh thu toàn Tập đoàn ước đạt 149,6 nghìn tỷ đồng, vượt 28%, tăng 19% so với cùng kỳ năm 2023; Nộp ngân sách toàn Tập đoàn ước đạt 20,6 nghìn tỷ đồng, vượt 18%, tăng 5% so với cùng kỳ năm 2023.', 'Theo đó, tổng doanh thu phí bảo hiểm gốc của thị trường phi nhân thọ 3 tháng đầu năm 2024 ước đạt 20.012 tỷ đồng, tăng 12,25% so với cùng kỳ năm 2023.\nDẫn đầu thị trường về doanh thu phí gốc là PVI với doanh thu ước đạt 4.217 tỷ đồng, tăng 30,53% so với cùng kỳ năm 2023, chiếm thị phần 21,07%.\nTiếp đến là Bảo Việt với doanh thu ước đạt 2.656 tỷ đồng, tăng 1.80% so với cùng kỳ năm 2023, chiếm 

In [9]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
def answer(question,context,embeddings):
    # model='gemini-2.0-flash'
    model='gemini-2.5-flash'
    llm=ChatGoogleGenerativeAI(model=model)
    context_ed= retrieve_with_energy_clustering(question,context,embeddings)

    if not context_ed:
        return 'Khong co thong tin'

    template='''Dựa vào các đoạn văn sau:

 {context}

 Trả lời câu hỏi: {query}

 YÊU CẦU BẮT BUỘC:
 - Trả lời NGẮN GỌN, đi thẳng vào vấn đề
 - KHÔNG lặp lại câu hỏi trong câu trả lời
 - KHÔNG bắt đầu bằng "Dựa vào đoạn văn..."
 - Trích dẫn ĐẦY ĐỦ tất cả số liệu liên quan (%, tỷ đồng, mức tăng/giảm, mức đích...)
 - Nếu có nhiều thông tin liên quan, tổng hợp HẾT
 - Nếu không có thông tin, trả lời: "Không có thông tin về..."

 Câu trả lời:'''
    prompt=ChatPromptTemplate.from_template(template)
    chain = prompt | llm | StrOutputParser()

    return chain.invoke({'context' : context_ed,
                        'query':question                    
    })

In [11]:
# y='So sánh tốc độ tăng trưởng giữa doanh thu và lợi nhuận của Hòa Phát năm 2025?'
# tl=answer(y,retriever_cosine,embeddings)
# tl

In [ ]:
# while True:
#     q=input('nhap cau hoi')
#     if q.strip().lower() == "quit":        
#         break
#     else:
#         tl=answer(q,retriever_cosine,embeddings)
#         print(f'cau tra loi: {tl}')